# Dokumentacja narzędzia do automatyzacji rozgrywki w Margonem

## Podstawowe założenia i cele
- Projekt w swoim zamyśle ma na celu być jak najbezpieczniejszym narzędziem do automatyzacji rozgrywki, dalej zwany botem, macro. Poprzez zastosowanie rozwiązań opierających się na naśladowaniu gracza, tj. klikania myszką i klawiaturą, zachowujemy się bardziej jak człowiek, niż jak bot, co zmniejsza naszą szanse na wykrycie niepożądanych inputów przez grę. Większość botów korzysta z wyciągania informacji bezpośrednio ze strony (co zwiększa podejrzany traffic na stronie) i na tej podstawie podejmuje działania, ten projekt natomiast opiera się na tym co widzi na ekranie. <br>
- Bot ma na celu zdobywanie przedmiotów poprzez pokonywanie przeciwników, a następnie je sprzedaje u handlarza i powtarza cykl.

In [13]:
from func import *
from const import *
from visualization import *

## Czas pomiędzy akcjami gracza


In [14]:
create_hist(100000)

TypeError: type numpy.ndarray doesn't define __round__ method